# Introduction

The goal of this document is to construct a computer from logical gates. The basic logical gates that are used to construct the computer are: `And`, `Or`, and `Not`. The major outline of this document is followed from the book _The Elements of Computing Systems, Noam Nisan and Shimon Shocken (2008)_.

The book is followed from Boolean Logic (chapter 1) to Assembler (chapter 6). There are a total of 13 chapters. The construction of the Virtual Machine, and eventually an Operating System, is left out of scope. The main reason for this is because Python does not allow an easy 'screen' interface. The entire emulator will be written as a `REPL` application, which allows the user excessive control of the entire emulator. The cost of this, however, is that everything is done at a very low level. 

The goal is to create a computer that can run Assembly code, and that we have an assembler. I haven't followed the Assembly implementation in the book, instead, I used it as a baseline, and transformed it into my own preferences.

Website's that can be useful for further understanding:

 * http://fourier.eng.hmc.edu/e85_old/lectures/digital_logic/node6.html

# Boolean logic

Every Boolean function, no matter how complex, can be expressed using three Boolean operators only: `And`, `Or`, and `Not`.

## Gate logic

A _gate_ is a physical device that implements a Boolean function. If a Boolean function $f$ operates on $n$ variables and returns $m$ binary results, the gate that implements $f$ will have _$n$ input pins and $m$ output pins_. Today, most gates are implemented as transistors etched in silicon, packaged as _chips_. 

There are the following three _primitive gates_, which will be used to create all the other components. The gates are: `And`, `Or`, and `Not`. The following symbols are used to indicate each gate:

![Primitive gates](gates1.PNG)

We can define their Boolean functions mathematically as:

 1. $\text{And}(a,b)=a\land b$
 2. $\text{Or}(a,b)=a\lor b$
 3. $\text{Not}(a)=\lnot a$
 

For ease of writing, we will also adopt the convention to write a $1$ when the value is `True`, and $0$ if the value is `False`.

## Primitive gates

We will now implement the primitive logic gates in Python. The input variables can be either a Boolean value, or a function. The input will be called by the `boolf(x)` function, which returns a Boolean value, or executes the function. Allowing functions gives us the option to easily chain multiple gates together with references.

In [1]:
def boolf(x):
    if callable(x): return x()
    if type(x) is bool: return x
    raise ValueError('Unsupported type {} for boolf()'.format(type(x)))

In [2]:
class And():
    def __init__(self, a, b): self.a = a; self.b = b
    def out(self): return boolf(self.a) and boolf(self.b)
    def __repr__(self): return '{}'.format(self.out())

class Or():
    def __init__(self, a, b): self.a = a; self.b = b
    def out(self): return boolf(self.a) or boolf(self.b)
    def __repr__(self): return '{}'.format(self.out())
    
class Not():
    def __init__(self, a): self.a = a
    def out(self): return not boolf(self.a)
    def __repr__(self): return '{}'.format(self.out())

To test a simple scenario, let $G(a,b,c) = \text{Not}(\text{Or}(a, \text{And}(b, c)))$. Because this is a composite gate, we are only using primitive gates. If $(a,b,c)=(0,1,1)$, the resulting output should be $0$.

In [3]:
a=False;b=c=True
G = Not(Or(lambda: a, And(lambda: b, lambda: c).out).out)
print(G)

b=False
print(G)

False
True


**Reference vs. value types**

Notice that primitive Boolean values should be the result of a function. If a Boolean value is the input for a gate, Python will reference to it by value, which is something we do not want. Thus, we input a function into the gate, and Python passes the function as a reference. Then we use `boolf(x)` to either get the variable, or return the function result. This allows us to change the input values, after the gate has been constructed, and keep calling `.out()` on the last gate to execute all the logic in the intermediate gates.

## Composite gates

We start with primitive gates and design more complicated functionality by interconnecting them, leading to the construction of _composite gates_.

### Three-way And gate

The first gate we can construct from two And gates, is a three-way And gate. It can be defined in the following way: $\text{And3W}(a,b,c)= \text{And}(\text{And}(a,b),c)$.

![Three-way And gate diagram](gates2.PNG)

In [4]:
class And3W():
    def __init__(self, a, b, c): self.a = a; self.b = b; self.c = c
    def out(self): 
        and1 = And(boolf(self.a), boolf(self.b))
        and2 = And(and1.out, boolf(self.c))
        return and2.out()
    def __repr__(self): return '{}'.format(self.out())

In [5]:
And3W(True, True, True)

True

### Xor gate

Let us consider another logic design example -- that of a `Xor` gate. The gate $\text{Xor}(a,b)$ is $1$ exactly when $a=1$ and $b=0$, or $a=0$ and $b=1$. In the case where $a=b$ the gate is $0$. It can be defined in the following way: $\text{Xor(a,b)}=\text{Or}(\text{And}(a,\text{Not}(b)), \text{And}(\text{Not}(a),b))$. This definition leads to the logic design shown in the figure below:

In [6]:
class Xor():
    def __init__(self, a, b): self.a = a; self.b = b
    def out(self): return Or(And(boolf(self.a), Not(boolf(self.b)).out).out,\
                             And(Not(boolf(self.a)).out, boolf(self.b)).out\
                          ).out()
    def __repr__(self): return '{}'.format(self.out())

In [7]:
Xor(False, True).out() and Xor(True, False).out() \
    and not Xor(True, True).out() and not Xor(False, False).out()

True

## Basic Logic Gates

In this section we will define all the basic logical gates that are used as the building blocks to create more advanced circuits.

### Not 

A not gate has a single input, and inverts that input as output. It is defined as $\text{Not}(a)=\lnot a$. The following truth table applies for a Not gate:

|in|out|
|--|--|
|0|1|
|1|0|

The gate has already been implemented earlier in _Primitive gates_.

### And

An And gate has two inputs $a, b$, and only outputs a $1$ if exacty $a=b=1$. It is defined as $\text{And}(a,b)=a \land b$. The following truth table applies for an And gate:

|a|b|out|
|--|--|--|
|0|0|0|
|0|1|0|
|1|0|0|
|1|1|1|

The gate has already been implemented earlier in _Primitive gates_.

### Or

An Or gate has two inputs $a, b$, and only outputs a $0$ if exacty $a=b=0$, otherwise it will output $1$. It is defined as $\text{Or}(a,b)=a \lor b$. The following truth table applies for an Or gate:

|a|b|out|
|--|--|--|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|1|

The gate has already been implemented earlier in _Primitive gates_.

### Xor

An Xor gate has two inputs $a,b$, its output is $1$ exactly when $a=1$ and $b=0$, or $a=0$ and $b=1$. Otherwise, the output is $0$. It is defined as $\text{Xor(a,b)}=\text{Or}(\text{And}(a,\text{Not}(b)), \text{And}(\text{Not}(a),b))$. The following thruth table applies for a Xor gate:

|a|b|out|
|--|--|--|
|0|0|0|
|0|1|1|
|1|0|1|
|1|1|0|

### Nand

A Nand gate has two input $a,b$ and its output is the inverse of an And gate. It's value is always $1$, except when $a=b=1$ for which the output is $0$. It is defined as $\text{Nand}(a,b)=\text{Not}(\text{And}(a,b))$. The following truth table applies for a Nand gate:

|a|b|out|
|--|--|--|
|0|0|1|
|0|1|1|
|1|0|1|
|1|1|0|

In Python we can implement the gate in the following way:

In [8]:
class Nand():
    def __init__(self,a,b): self.a=a; self.b=b
    def out(self): return Not(And(boolf(self.a), boolf(self.b)).out).out()
    def __repr__(self): return '{}'.format(self.out())

In [12]:
Nand(False, False).out() and Nand(True, False).out()\
    and Nand(False, True).out() and not Nand(True, True).out()

True

### Multiplexor

A multiplexor is a three-input gate that uses one of the inputs, called _selection bit_, to select and output one of the other two inputs, called _data bits_. It is defined as $\text{Mux}(a,b,sel)=\text{Or}(\text{And}(a, \text{Not}(sel), \text{And}(b,sel))$. The following truth table applies for a multiplexor:

|a|b|sel|out|
|--|--|--|--|
|0|0|0|0|
|0|1|0|0|
|1|0|0|1|
|1|1|0|1|
|0|0|1|0|
|0|1|1|1|
|1|0|1|0|
|1|1|1|1|

Which can also be expressed in a more simple form, such as:

|sel|out|
|--|--|
|0|a|
|1|b|

Using the definition, we can implement it in the following way:

In [13]:
class Mux():
    def __init__(self,a,b,sel): self.a=a; self.b=b; self.sel=sel
    def out(self): return Or(And(boolf(self.a), Not(boolf(self.sel)).out).out,\
                             And(boolf(self.b), boolf(self.sel)).out
                          ).out()
    def __repr__(self): return '{}'.format(self.out())

In [17]:
Mux(True, False, False).out() and Mux(False, True, True).out()\
    and not Mux(False, False, False).out()

True

### Demultiplexor

A demultiplexor performs the opposite function of a multiplexor: It takes a single input and channels it to one of two possible outputs according to a selector bit that specifies which output to choose. It is defined for each output bit as:

$$ \text{DMux}(in,sel) = \begin{cases}\begin{aligned} \text{And}(in,\text{Not}(sel)) \quad &\text{output bit } a \\ \text{And}(in,sel) \quad &\text{output bit } b \end{aligned}\end{cases} $$

It can be represented with the following table:

|sel|a|b|
|--|--|--|
|0|in|0|
|1|0|in|

Using the definition, it can be implement in Python in the following way. Notice that $z=in$ because `in` is a reserved keyword in Python.

In [18]:
class DMux():
    def __init__(self, z, sel): self.z=z; self.sel=sel;
    def a(self): return And(boolf(self.z), Not(boolf(self.sel)).out).out()
    def b(self): return And(boolf(self.z), boolf(self.sel)).out()
    def __repr__(self): return 'a: {}, b: {}'.format(self.a(), self.b())

In [19]:
DMux(True, False)

a: True, b: False

In [20]:
DMux(True, True)

a: False, b: True

## Multi-Bit Versions of Basic Gates

Computer hardware is typically designed to operate on multi-bit arrays called "buses". For example, a basic requirement of a 32-bit computer is to be able to compute (bit-wise) an And function of two given 32-bit buses. To implement this operation, we can build an array of 32 binary And gates, each operating seperately on a pair of bits. In order to enclose all this logic in one package, we can encapsulate the gates array in a single chip interface consistring of two 32-bit input buses and one 32-bit output bus.

This section describes a typical set of such multi-bit logic gates, as needed for the construction of a typical 16-bit computer. We note in passing that the architecture of _n_-bit logic gates is basically the same irrespective of _n_'s values.

Also, we will adopt the notation to refer to the individual bits. For example, to refer to individual bits in a 16-bit bus named `data`, we use the notation `data[0], data[1], ..., data[15]`.

We will also define a helper function `bstr(L)`, that transforms a list of Boolean values into a binary string. For example, the list `[True, True, False]` gives the binary string: `110`. This is useful because it gives a compact representation of each individual bit.

In [143]:
def bstr(L): 
    out=''; 
    for i in L: out += '1' if i else '0'
    return out

In [146]:
bstr([False,True,False,False,False,True]+10*[False])

'0100010000000000'

### Not16

In [120]:
class Not16():
    def __init__(self, in16): self.in16=in16
    def out(self): return [not self.in16[i] for i in range(16)]
    def __repr__(self): return bstr(self.out())

In [121]:
Not16([False] * 16)

1111111111111111

### And16

In [122]:
class And16():
    def __init__(self, a, b): self.a=a; self.b=b
    def out(self): return [self.a[i] and self.b[i] for i in range(16)]
    def __repr__(self): return bstr(self.out())

In [123]:
And16([True]*16, [False]*15+[True])

0000000000000001

### Or16

In [124]:
class Or16():
    def __init__(self, a, b): self.a=a; self.b=b
    def out(self): return [self.a[i] or self.b[i] for i in range(16)]
    def __repr__(self): return bstr(self.out())

In [125]:
Or16([False]+15*[True], [False]*15+[True])

0111111111111111

In [139]:
bits=16
x = [False]*bits
y = [False]*bits
x[1]=y[1]=True
F = And16(Or16(x,y).out(), y)
print(bstr(F.a))
x[2]=y[2]=True

#Required to manually update a, b (Boooh!)
F.a=x;F.b=y; 
print(bstr(F.a))

0100000000000000
0110000000000000


In [140]:
a=x
print(bstr(a))
# Now we have reference types, because a and x are lists.
# Lists are classes, and those are passed on by reference.
x[2]=True
print(bstr(a))

0110000000000000
0110000000000000


### Mux16

## Multi-Way Versions of Basic Gates

### Or8Way

### Multi-Way/Multi-Bit Multiplexor

#### Mux4Way16

#### Mux8Way16

#### DMux4Way

#### DMux8Way

# Boolean Arithmetic

# Sequential Logic

# Machine Language

# Computer Architecture

# Assembler